<a href="https://colab.research.google.com/github/STRIDES-Codes/NL4Cell-Integrating-NLP-with-single-cell-data-analysis/blob/main/CellAttention_with_RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
%%capture
!pip install git+https://github.com/justinphan3110/transformers.git

In [ ]:
from transformers import EncoderDecoderModel, RobertaTokenizer, RobertaConfig, RobertaModel
import torch
import numpy as np

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
  print(torch.cuda.get_device_name())
else:
  device = torch.device('cpu')
  

Tesla V100-SXM2-16GB


## Train

In [ ]:
!pip install tokenizers

In [ ]:
!gsutil cp gs://cytereader/preprocessed_cell_corpus_0.txt . 
!gsutil cp gs://cytereader/preprocessed_cell_corpus_1.txt . 

Copying gs://cytereader/preprocessed_cell_corpus_0.txt...
\ [1 files][267.0 MiB/267.0 MiB]                                                
Operation completed over 1 objects/267.0 MiB.                                    
Copying gs://cytereader/preprocessed_cell_corpus_1.txt...
\ [1 files][267.0 MiB/267.0 MiB]                                                
Operation completed over 1 objects/267.0 MiB.                                    


In [ ]:
from tokenizers import BertWordPieceTokenizer

!mkdir cellAttention
wb_tokenizer = BertWordPieceTokenizer(clean_text=True,
                                      strip_accents=True, lowercase=False)

wb_tokenizer.train(['preprocessed_cell_corpus_0.txt'],
                   vocab_size=10000, min_frequency=2,
                   special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])
wb_tokenizer.save_model(".")

KeyboardInterrupt: ignored

In [ ]:
from transformers import RobertaConfig, BertTokenizer, RobertaModel, RobertaForMaskedLM, RobertaForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('cellAttention', max_len=64)
configuration = RobertaConfig(vocab_size=1000)


model = RobertaForMaskedLM(configuration)

Didn't find file cellAttention/added_tokens.json. We won't load it.
Didn't find file cellAttention/special_tokens_map.json. We won't load it.
Didn't find file cellAttention/tokenizer_config.json. We won't load it.
Didn't find file cellAttention/tokenizer.json. We won't load it.
loading file cellAttention/vocab.txt
loading file None
loading file None
loading file None
loading file None


In [ ]:
model.num_parameters()

86810882

In [ ]:
cnt = 0
with open('small.txt', 'w') as file:
  for line in open('preprocessed_cell_corpus_0.txt'):
    file.write(line)
    cnt += 1
    if cnt == 100000:
      break

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="small.txt",
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 2min 20s, sys: 1.15 s, total: 2min 21s
Wall time: 2min 21s


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./cellAttention",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# %%time
trainer.train()

***** Running training *****
  Num examples = 100000
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1563


Step,Training Loss
500,0.447900
1000,0.331700
1500,0.316800




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1563, training_loss=0.3633012033317307, metrics={'train_runtime': 537.8634, 'train_samples_per_second': 185.921, 'train_steps_per_second': 2.906, 'total_flos': 5241726633600000.0, 'train_loss': 0.3633012033317307, 'epoch': 1.0})

In [ ]:
trainer.save_model("./cellAttention")

Saving model checkpoint to ./cellAttention
Configuration saved in ./cellAttention/config.json
Model weights saved in ./cellAttention/pytorch_model.bin


In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./cellAttention",
    tokenizer="./cellAttention"
)

loading configuration file ./cellAttention/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.9.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 1000
}

loading configuration file ./cellAttention/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden

OSError: ignored

In [ ]:
fill_mask("CD45+ CD196_CCR6+ CD181_CXCR1- HLA_DR- CD15- CD31_PECAM1- CD8a- CD182_CXCR2[MASK] CD66ace- CD63- CD14- CD66b- CD62L_Lselectin- CD3+ CD27- CD86+ CD10- CD197_CCR7+ CD28- CD11c- CD33- CD161- CD45RO- CD24- CD38+ CD278_ICOS- CD32- CD152_CTLA4+ IgM+ CD184_CXCR4+ CD279_PD1- CD56+ CD16-")

In [ ]:
input = 'CD45+ CD196_CCR6+ CD181_CXCR1- HLA_DR- CD15-'
masked_input = 'CD45+ CD196_CCR6+ CD181_CXCR1[MASKED] HLA_DR- CD15-'

output = 'CD45+ CD196_CCR6+ CD181_CXCR1-/+ HLA_DR- CD15-'